In [14]:
import requests
import os
import json
from dotenv import load_dotenv
import pathlib
import pandas as pd
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

# name the file we want
env_file_name = 'lrhf97_twit_cred.env'

# designate the path of the file
curr_file = pathlib.Path.cwd()

# join them
env_path = curr_file.joinpath(env_file_name)

# load the file
load_dotenv(env_path)

bearer_token = os.getenv("BEARER_TOKEN")


def create_url():
    # Replace with user ID below - RidgeSupply =3413455348
    user_id = 3413455348
    max_users = 10
    
    return "https://api.twitter.com/2/users/{}/followers?max_results={}".format(user_id,max_users)

def get_params():
    return {'user.fields': 'id,name,username,created_at,description,public_metrics,verified,location',
            'next_token': {}}
    # return {'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
    #         'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
    #         'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
    #         'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
    #         'next_token': {}}

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FollowersLookupPython"
    return r


def connect_to_endpoint(url, params, next_token=None):
    params['next_token'] = next_token # param object recieved form the create_url function
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url = create_url()
    params = get_params()
    json_response = connect_to_endpoint(url, params,)
    # print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response


ridge_followers= main()

200


In [15]:
df=pd.DataFrame.from_dict(ridge_followers['data'])
df

,name,verified,description,id,username,created_at,public_metrics,location
0,J,False,"Husband, dad, cyclist and a F'n 🦧",3386288339,DumbMoney_AMC,2015-07-21T20:02:19.000Z,"{'followers_count': 2, 'following_count': 72, ...",NaN
1,frankgiraffe,False,Living at the intersection of geekery and the ...,14387509,frankgiraffe,2008-04-14T16:56:54.000Z,"{'followers_count': 215, 'following_count': 37...","SF, CA USA"
2,Jordan Spoon,False,Slinging paints on cars for @maconisetupshop -...,33535739,jspoon13,2009-04-20T14:52:11.000Z,"{'followers_count': 85, 'following_count': 606...","Kernersville, NC"
3,f3morganton,False,"Fitness, Fellowship, & Faith coming to Morgant...",1418283365577461764,f3morganton,2021-07-22T18:55:59.000Z,"{'followers_count': 22, 'following_count': 78,...",NaN
4,DSRPT cycling,False,,1416820910712135682,CyclingDsrpt,2021-07-18T18:03:36.000Z,"{'followers_count': 0, 'following_count': 47, ...",NaN
5,Steven Boyd,False,"@UChicago polisci grad student - IR, formal th...",1268533564196478987,boyd_steven_s,2020-06-04T13:24:01.000Z,"{'followers_count': 40, 'following_count': 281...",NaN
6,Daniel De Nobrega,False,Caraqueño,52964262,dandng,2009-07-02T04:08:48.000Z,"{'followers_count': 444, 'following_count': 16...","Manchester, England"
7,Timothy Anderson,False,"Rider of bikes, and drinker of beer. I like to...",385446082,tanderson999,2011-10-05T14:27:33.000Z,"{'followers_count': 118, 'following_count': 75...",NaN
8,velokicks,False,Have a weakness for shoes? So do we.,877695112884633601,velokicks,2017-06-22T01:09:56.000Z,"{'followers_count': 294, 'following_count': 23...","Melbourne, Victoria"
9,Wookie,False,🚴 ☕️ 🎮 🥘 📷 🪴 🥖 #adhd,1162847621225910272,wookieapproved,2019-08-17T22:04:22.000Z,"{'followers_count': 97, 'following_count': 210...","Denver, CO"


In [16]:
df.public_metrics[0]

{'followers_count': 2,
 'following_count': 72,
 'tweet_count': 28,
 'listed_count': 0}

In [17]:
ridge_followers['meta']['next_token']

'GOMK7O8T8HUHEZZZ'

In [33]:
import csv
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['id', 'created_at', 'location', 'description','username','name','verified', 'followers_count', 'following_count', 'tweet_count','listed_count'])
csvFile.close()

In [34]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. ID
        id = tweet['id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('location' in tweet):   
            location = tweet['location']
        else:
            location = " "

        # 4. description
        description = tweet['description']

        # # 5. Language
        username = tweet['username']
        name = tweet['name']
        verified = tweet['verified']


        # 6. Tweet metrics
        followers_count = tweet['public_metrics']['followers_count']
        following_count = tweet['public_metrics']['following_count']
        tweet_count = tweet['public_metrics']['tweet_count']
        listed_count = tweet['public_metrics']['listed_count']


        # # 7. source
        # source = tweet['source']

        # # 8. Tweet text
        # text = tweet['text']
        
        # Assemble all data in a list
        res = [id, created_at, location, description,username,name,verified, followers_count, following_count, tweet_count,listed_count]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Followers added from this response: ", counter) 

In [35]:
append_to_csv(ridge_followers,'data.csv')

# of Followers added from this response:  10
